# 朴素贝叶斯与应用

## 1.贝叶斯理论简单回顾

在我们有一大堆样本（包含**特征**和**类别**）的时候，我们非常容易通过统计得到$p(特征|类别)$。

$$p(x)p(y|x) = p(y)p(x|y)$$

变化公式得到：
$$p(特征)p(类别|特征) = p(类别)p(特征|类别)$$$$p(类别|特征) = \frac{p(类别)p(特征|类别)}{p(特征)}$$

## 2.独立假设

<p>看起来很简单，但实际上，你的特征可能是很多维的</p>
$$p(features|class) = p({f_0, f_1, \ldots ,f_n}|c)$$<p>就算是2个维度吧，可以简单写成</p>
$$p({f_0, f_1}|c) = p(f_1|c, f_0)p(f_0|c)$$<p>这时候我们加一个特别牛逼的假设：特征之间是独立的。这样就得到了</p>
$$p({f_0, f_1}|c) = p(f_1|c)p(f_0|c)$$<p>其实也就是：</p>
$$p({f_0, f_1, \ldots, f_n}|c) = \Pi^n_i p(f_i|c)$$

## 3.贝叶斯分类器

<p>OK，回到机器学习，其实我们就是对每个类别计算一个概率$p(c_i)$，然后再计算所有特征的条件概率$p(f_j|c_i)$，那么分类的时候我们就是依据贝叶斯找一个最可能的类别：</p>
$$p(class_i|{f_0, f_1, \ldots, f_n})= \frac{p(class_i)}{p({f_0, f_1, \ldots, f_n})} \Pi^n_j p(f_j|c_i)$$

## 4. 文本分类问题

下面我们来看一个文本分类问题，经典的新闻主题分类，用朴素贝叶斯怎么做。

In [3]:
# coding:utf-8
import os
import time
import random
import jieba
import nltk
import sklearn
from sklearn.naive_bayes import MultinomialNB
import numpy as np
import pylab as pl
import matplotlib.pyplot as plt

In [ ]:
def make_word_set(words_file):
    '''
    
    '''
    words_set = set()
    with open(words_file, 'r') as fp:
        for line in fp.readlines():
            